In [2]:
import xarray as xr
import pandas as pd
import glob

In [3]:
base_path = "/gws/ssde/j25b/eds_ai/frame-fm/data/"
data_path = "inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/"

In [4]:
daily = xr.Dataset()


In [ ]:
# Descriptions of metadata are in cosmos-uk_supportinginformation_2013-2024.docx from https://data-package.ceh.ac.uk/sd/2dce161d-2fab-47bb-9fe6-38e7ed1ae18a.zip

In [5]:
%%time
files = glob.glob(base_path + data_path + "cosmos-uk_*_hydrosoil_sh_????-????.csv")

dfs = []
#load each CSV into the dfs array
for file in files:
    print(file)
    df = pd.read_csv(file, delimiter=",",parse_dates=["DATE_TIME"],na_values=[-9999])
    
    #remove timezone from the datetime as xarray's to_netcdf doesn't like it
    df["DATE_TIME"] = pd.to_datetime(df.DATE_TIME).dt.tz_localize(None)
    #make DATE_TIME the index instead of using a index number
    dfs.append(df.set_index("DATE_TIME"))
    #dfs.append(df)

#sorts all the entries by date_time as not doing can break selecting on dates
#note that there can be multiple entries with the same timestamp so the index_col entries are not unique
ds = pd.concat(dfs).sort_index().to_xarray() 

/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/cosmos-uk_alic1_hydrosoil_sh_2015-2024.csv
/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/cosmos-uk_balrd_hydrosoil_sh_2014-2024.csv
/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/cosmos-uk_bickl_hydrosoil_sh_2015-2024.csv
/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/cosmos-uk_bunny_hydrosoil_sh_2015-2024.csv
/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/cosmos-uk_cardt_hydrosoil_sh_2015-2024.csv
/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/daily_and_subdaily_hydrometeorological_and_soil_moisture_data/data/cosmos-uk_cgarw_hydrosoil_sh_2016-2024.csv
/gws/ssde/j25b/eds_ai/frame-fm/data/inputs/daily_and_subdaily_hydrometeorological_

In [7]:
ds

<xarray.Dataset> Size: 3GB
Dimensions:         (DATE_TIME: 8299791)
Coordinates:
  * DATE_TIME       (DATE_TIME) datetime64[ns] 66MB 2013-10-02T13:30:00 ... 2...
Data variables: (12/46)
    SITE_ID         (DATE_TIME) object 66MB 'CHIMN' 'CHIMN' ... 'ELMST' 'WYTH1'
    LWIN            (DATE_TIME) float64 66MB nan nan nan nan ... 325.3 337.4 nan
    LWOUT           (DATE_TIME) float64 66MB nan nan nan nan ... 342.4 366.2 nan
    SWIN            (DATE_TIME) float64 66MB nan nan nan ... -1.24 -1.412 nan
    SWOUT           (DATE_TIME) float64 66MB nan nan nan ... 0.136 -0.041 nan
    RN              (DATE_TIME) float64 66MB nan nan nan ... -18.48 -30.17 nan
    ...              ...
    TDT10_VWC       (DATE_TIME) float64 66MB nan nan nan nan ... 58.96 41.05 nan
    STP_TSOIL2      (DATE_TIME) float64 66MB nan nan nan nan ... 7.771 7.678 nan
    STP_TSOIL5      (DATE_TIME) float64 66MB nan nan nan nan ... 7.919 7.633 nan
    STP_TSOIL10     (DATE_TIME) float64 66MB nan nan nan nan ... 8.05 7.609 nan
    STP_TSOIL20     (DATE_TIME) float64 66MB nan nan nan nan ... 8.17 7.634 nan
    STP_TSOIL50     (DATE_TIME) float64 66MB nan nan nan nan ... 8.15 8.1 nan

In [8]:
# free up some memory, by getting rid of the Pandas version of the data
# frees about 2GB
dfs = None

In [9]:
ds.sel(DATE_TIME=slice("2015-05-01","2015-05-02"))

<xarray.Dataset> Size: 1MB
Dimensions:         (DATE_TIME: 2688)
Coordinates:
  * DATE_TIME       (DATE_TIME) datetime64[ns] 22kB 2015-05-01 ... 2015-05-02...
Data variables: (12/46)
    SITE_ID         (DATE_TIME) object 22kB 'ALIC1' 'CHOBH' ... 'LULLN' 'CRICH'
    LWIN            (DATE_TIME) float64 22kB 271.1 274.7 277.2 ... 366.9 332.5
    LWOUT           (DATE_TIME) float64 22kB 337.5 334.2 320.4 ... 367.6 337.8
    SWIN            (DATE_TIME) float64 22kB -3.355 -3.517 ... -0.295 -1.612
    SWOUT           (DATE_TIME) float64 22kB 1.501 1.188 1.884 ... 0.208 1.577
    RN              (DATE_TIME) float64 22kB -71.26 -64.2 ... -1.203 -8.489
    ...              ...
    TDT10_VWC       (DATE_TIME) float64 22kB nan nan nan nan ... nan nan nan nan
    STP_TSOIL2      (DATE_TIME) float64 22kB 7.535 8.76 7.635 ... 10.79 6.471
    STP_TSOIL5      (DATE_TIME) float64 22kB 8.51 9.06 8.56 ... 10.26 10.76 6.7
    STP_TSOIL10     (DATE_TIME) float64 22kB 9.13 9.41 9.54 ... 10.72 6.957
    STP_TSOIL20     (DATE_TIME) float64 22kB 9.31 9.78 10.49 ... 10.49 7.265
    STP_TSOIL50     (DATE_TIME) float64 22kB 8.9 9.37 10.65 ... 9.51 9.95 7.58

In [10]:
ds.to_netcdf("/gws/ssde/j25b/eds_ai/frame-fm/users/colinsau/cosmos-uk-hydrosoil.nc")